In [336]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric, ICRS, Galactic
import gala.coordinates as gc
import sys, time, mpmath
import statsmodels.stats.api as sms

## Input Parameters

In [287]:
ra = 168.3700 #degrees
dec = 22.15167 #degrees
distance = 233 #kpc
mu_a = -6.930000 #mas/cent
mu_d = -8.670000 #mas/cent
rv = 79.1 #km/s

#Uncertainty in the proper motion components
#These are taken as the std. devs. for the Gaussian distributions for the MC
mu_a_std = 3.73  
mu_d_std = 3.91
rv_std = 0.6

#luminosity of the dSph and the min and max M/L
ldsph = 1.e3
mol1 = 1.
mol2 = 3.

## Functions to Get Positions and Velocities

In [316]:
#Constants
G = 4.498933261e-6 #Gravitational constant in kpc^3/(M_sun Gyr)
vcon = 1.0226903 #Conversion factor from kpc/Gyr to km/s (i.e. multiply by km/s to get kpc/Gyr)
asec_to_rad = 206264.8 #Converts arcseconds to radians
ratoam = 3437.747 #Converts radians to arcmin
year_to_sec = 3.155693e7 #Converts from years to seconds
pc_to_km = 3.08568e13 #Converts from pc to km
year_to_day = 365.2422 #Converts years to days
usun = -10.0 #U-component of Sun's velocity w.r.t. LSR; Dehnen & Binney                  
vsun = 11.0 #V-component of Sun's velocity w.r.t. LSR; 1998, MNRAS, 
wsun = 7.0 #W-component of Sun's velocity w.r.t. LSR; 298, 387
vlsr = 237.0 #Circular velocity of LSR w.r.t. Galaxy center in km/s
dlsr = 8.2 #Distance of LSR from the Galaxy center in pc

def sin_deg(x):
    #Sine function for degree-valued args
    return np.sin(np.deg2rad(x))
def cos_deg(x):
    #Cosine function for degree-valued args
    return np.cos(np.deg2rad(x))

def radec_to_lb(ra, dec):
    #Returns galactic coords (l, b) in degrees, given an object's RA/DEC in degrees
    coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    galactic = coords.galactic
    return galactic.l.deg, galactic.b.deg

def lb_to_galactic_xyz(l, b, d, dlsr=dlsr):
    #Takes l and b in degrees, d (distance to object) in kpc, dlsr in kpc
    x = (dlsr-d*cos_deg(b)*cos_deg(l))
    y = -d*cos_deg(b)*sin_deg(l)
    z = d*sin_deg(b)
    return x, y, z #kpc

def pm_radec_to_lb(ra, dec, dist, mu_a, mu_d, rv):
    #Takes ra/dec in degrees, distance in pc, mu_a/mu_d in mas/cent, rv in km/s. Returns mu_l/mu_b in mas/cent
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.pc)
    pm = [mu_a, mu_d] * u.mas/(100*u.yr) 
    
    result = gc.pm_icrs_to_gal(c, pm).to(u.mas/(u.yr))*u.yr/u.mas
    mu_l = float(result[0]*100)
    mu_b = float(result[1]*100)
    
    return mu_l, mu_b #mas/cent

def pm_lb_to_galactic_vt(mu_a,mu_d,d,asec_to_rad=asec_to_rad,year_to_sec=year_to_sec,pc_to_km=pc_to_km):
    #mul, mub in mas/cent, distance in kpc
    #Converts l,b pm into tangential velocity in l-direction and b-direction,
    #and gets magnitude vt of tangential velocity
    
    #Get into arcsec/year
    mu_a = (mu_a*(1e-3)/100)
    mu_d = (mu_d*(1e-3)/100)

    vtl=mu_a*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vtb=mu_d*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vt=np.sqrt(vtl*vtl+vtb*vtb)
    return vtl, vtb, vt  

def galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, v_r_sun=rv, usun=usun, vsun=vsun, wsun=wsun):
    #Finds velocity of galaxy wrt LSR. velocity inputs are in km/s
    #Theta direction
    vg = v_r_sun*cos_deg(b)*sin_deg(l) - vtb*sin_deg(b)*sin_deg(l)+vtl*cos_deg(l) + vsun
    #Pi direction
    ug = -v_r_sun*cos_deg(b)*cos_deg(l) + vtb*sin_deg(b)*cos_deg(l) + vtl*sin_deg(l) + usun
    #Z direction
    wg =  v_r_sun*sin_deg(b) + vtb*cos_deg(b) + wsun
    
    return ug, vg, wg #km/s

def v_uvg_to_cyl(l,b,d,ug,vg,wg, dlsr=dlsr, vlsr=vlsr):
    #input velocities in km/s, distances in kpc
    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    vpi =  ug*cosalpha + (vg+vlsr)*sinalpha
    vtheta = -ug*sinalpha + (vg+vlsr)*cosalpha
    vz =  wg
    return vpi, vtheta, vz #km/s

def v_cyl_to_xyz(l, b, d, vpi, vth, vz, dlsr=dlsr):
    #distances in kpc, vels in km/s

    x=dlsr-d*cos_deg(b)*cos_deg(l)
    y=-d*cos_deg(b)*sin_deg(l)
    z=d*sin_deg(b)

    costh=x/np.sqrt(x**2+y**2)
    sinth=-y/np.sqrt(x**2+y**2)
    vx=vpi*costh-vth*sinth
    vy=-vpi*sinth-vth*costh
    return vx, vy, vz

def get_v_spherical(l, b, d, vpi, vtheta, vz, dlsr=dlsr):

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    vr =  vpi*cosbeta + vz*sinbeta
    vrot = vtheta
    v_perp_radial = -vpi*sinbeta + vz*cosbeta
    vt = np.sqrt(vrot**2+v_perp_radial**2)
    
    return vr, vrot, v_perp_radial, vt

def get_vxvyvz(ra, dec, distance, mu_a, mu_d, rv):
    l, b = radec_to_lb(ra, dec)
    x, y, z = lb_to_galactic_xyz(l, b, distance)
    
    mu_l, mu_b = pm_radec_to_lb(ra, dec, distance, mu_a, mu_d, rv)
    vtl, vtb, vt = pm_lb_to_galactic_vt(mu_l, mu_b, distance)    
    ug, vg, wg = galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, rv)
    vpi, vtheta, vz = v_uvg_to_cyl(l, b, distance, ug, vg, wg)
    vx, vy, vz = v_cyl_to_xyz(l, b, distance, vpi, vtheta, vz)
    
    return vx, vy, vz

def get_xyz(ra, dec, distance):
    l, b = radec_to_lb(ra, dec)
    x, y, z = lb_to_galactic_xyz(l, b, distance)
    return x, y, z

print get_vxvyvz(ra, dec, distance, mu_a, mu_d, rv)
print get_xyz(ra, dec, distance)
l, b = radec_to_lb(ra, dec)

(41.227183073492441, -115.55907658317186, 40.498068962037358)
(77.105875011103919, 58.166385352063067, 214.84331966349205)


## Print Values

In [301]:
l, b = radec_to_lb(ra, dec)
print "l = {0}, b = {1}" .format(l,b) +" (deg)" + "\n"

x0, y0, z0 = [i for i in lb_to_galactic_xyz(l, b, distance)]
print "Galactocentric X, Y, Z = {0}, {1}, {2}" .format(x0, y0, z0) + " (kpc)" + "\n"

mu_l, mu_b = pm_radec_to_lb(ra, dec, distance, mu_a, mu_d, rv) #*u.yr/u.mas
print "mu_l = {0}, mu_b = {1}" .format (mu_l, mu_b) + " (mas/cent)" + "\n"

vtl, vtb, vt = pm_lb_to_galactic_vt(mu_l, mu_b, distance)
print "v_tang in l direction = {0}, v_tang in b direction = {1}, v_tang total = {2}".format(vtl, vtb, vt) \
+ " (km/s)" + "\n"

ug, vg, wg = galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, rv)
print "Velocities of galaxy wrt LSR:"
print "u = {0}, v = {1}, w = {2}" .format(ug, vg, wg) + " (km/s)" + "\n"

vpi, vtheta, vz = v_uvg_to_cyl(l,b,distance,ug,vg,wg)
print "Cylindrical velocities:"
print "vpi = {0}, vtheta = {1}, vz = {2}" .format(vpi, vtheta, vz) + " (km/s)" + "\n"

vx0, vy0, vz0 = v_cyl_to_xyz(l, b, distance, vpi, vtheta, vz)
print "Galactocentric XYZ velocities:"
print "Vx = {0}, Vy = {1}, Vz = {2}" .format (vx0, vy0, vz0) + " (km/s)" + "\n"

vr, vrot, v_perp_radial, vt = get_v_spherical(l, b, distance, vpi, vtheta, vz)
print "Radial velocity = {0}" .format(vr) + " (km/s)"
print "Velocity in direction of disk rotation = {0}" .format(vrot) + " (km/s)"
print "V up and perp to radial = {0}" .format(v_perp_radial) + " (km/s)"
print "Tangential velocity = {0}" .format(vt) + " (km/s)"

l = 220.16913737, b = 67.2312443585 (deg)

Galactocentric X, Y, Z = 77.1058750111, 58.1663853521, 214.843319663 (kpc)

mu_l = 6.17081529568, mu_b = -9.22577180982 (mas/cent)

v_tang in l direction = 68.1599852049, v_tang in b direction = -101.903628602, v_tang total = 122.597443307 (km/s)

Velocities of galaxy wrt LSR:
u = 41.2271830735, v = -121.440923417, w = 40.498068962 (km/s)

Cylindrical velocities:
vpi = -36.6806351847, vtheta = 117.081645903, vz = 40.498068962 (km/s)

Galactocentric XYZ velocities:
Vx = 41.2271830735, Vy = -115.559076583, Vz = 40.498068962 (km/s)

Radial velocity = 21.896955667 (km/s)
Velocity in direction of disk rotation = 117.081645903 (km/s)
V up and perp to radial = 50.0608222039 (km/s)
Tangential velocity = 127.334982338 (km/s)


# Orbit

## XENERGY, XFORCE

In [320]:
def gammq(a,x):
    #a-1 is power inside the integral
    #x = lower bound of integration
    #mpmath.gammainc defines them differently,
    #so we make the switch when calling it herein
    #mpmath.mp.dps = 15 # arbitrary precision!
    return float(mpmath.gammainc(z=a, a=x, regularized=True))

def XENERGY(x, y, z, vx, vy, vz, vcon=vcon, G=G):
    #bulge parameters
    mb=9.1e9
    rc=2.1
    #disk parameters
    #increase disk mass to get higher v_c at R=R_0.
    a=4.9e0
    b=0.30e0
    md=1.35e11
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2

    #The kinetic energy per unit mass
    ke=vcon*vcon*(vx*vx+vy*vy+vz*vz)/2.0
    
    #The potential energy per unit mass
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #disk
    phi1=-G*md/np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2)

    #bulge; this isn't right, but is accurate for r >> rc and that
    #is the region of interest for all or almost all of our range of
    #integration
    
    if r > rc/2.0:
        phi2 = -G*mb/r
    else:
        phi2 = -G*2.0*mb/rc
        
    #halo
    phi3 = -G*(mv/(np.log(1.0+c)-c/(1.0+c))) * np.log(1.0+r/ra)/r

    e=ke+phi1+phi2+phi3
    
    return e

def XFORCE(x, y, z, G=G):
    #bulge parameters
    mb=9.1e9
    rc=2.1
    #disk parameters
    #increase disk mass to get higher v_c at R=R_0.
    a=4.9
    b=0.30
    md=1.35e11
    
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2
    
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #bulge radial force: -GM_b(r)/r*r
    mb_r = mb*(1.-1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2, r/rc))
    #mb_r=mb*(1.00 - 1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2,r/rc))
    fb_r = -G*mb_r/(r*r)

    #NFW halo radial force: -GM_nfw(r)/r*r
    mnfw_r=mv*(np.log(1.0+r/ra)-r/(ra+r))/(np.log(1.0+c)-c/(1.0+c))
    fh_r = -G*mnfw_r/(r*r)

    #Find the X components of the force.
    fx1=-G*md*x/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fx2=x*fb_r/r
    fx3=x*fh_r/r
    fx=fx1+fx2+fx3

    #Find the Y components of the force.
    fy1=-G*md*y/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fy2=y*fb_r/r
    fy3=y*fh_r/r
    fy=fy1+fy2+fy3
    
    #Find the Z components of the force.
    fz1=-G*md*z/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fz1=fz1*(1.0+a/np.sqrt(z*z+b*b))
    fz2=z*fb_r/r
    fz3=z*fh_r/r
    fz=fz1+fz2+fz3

    return fx, fy, fz

In [330]:
#Create dictionaries which store references to force funtions and energy functions. 
#This gives an easy way to get fx, fy, fz and energies
#for any potential by changing a single parameter. 
#If we add later add force and energy functions for different potentials
#we can simply add their names to the dictionaries below and pass the
#'pot' argument in integrate_orbit() directly to these dictionaries.

forcefunctions = {'X':XFORCE} 
energyfunctions = {'X':XENERGY}

-15377.077287299018

# Integrate orbit

In [361]:
def integrate_orbit(pot, x0, y0, z0, vx0, vy0, vz0, vcon=vcon, t=t, NSTEP=NSTEP):
    
    dt = float(t)/float(NSTEP)
    
    #Initial angular momenta
    amx0=y0*vz0-z0*vy0
    amy0=z0*vx0-x0*vz0
    amz0=x0*vy0-y0*vx0
    am0=np.sqrt(amx0*amx0+amy0*amy0+amz0*amz0)
    bam0=np.arcsin(amz0/am0)
    lam0=np.arctan2(-amy0,-amx0)
    if(lam0 < 0.):
        lam0 += 2.*np.pi
        
    #Initial energy
    initialenergy = energyfunctions['X'](x0, y0, z0, vx0, vy0, vz0)
    
    dt = float(t)/float(NSTEP)
    x=x0
    y=y0
    z=z0
    r2=x**2+y**2+z**2
    vx=vx0
    vy=vy0
    vz=vz0
    
    times, positions = [[] for i in np.arange(2)] 
    timestep = 0
    times.append(timestep)
    positions.append((x,y,z,r2))

    fx, fy, fz = forcefunctions['X'](x, y, z)
    for i in np.arange(0, NSTEP): #-1)?
        timestep = timestep + dt
        vx=vx+0.5*dt*fx/vcon
        vy=vy+0.5*dt*fy/vcon
        vz=vz+0.5*dt*fz/vcon
        x=x+dt*vx*vcon
        y=y+dt*vy*vcon
        z=z+dt*vz*vcon
        r2 = x**2 + y**2 + z**2
        
        fx, fy, fz = forcefunctions['X'](x, y, z)
        vx=vx+0.5*dt*fx/vcon
        vy=vy+0.5*dt*fy/vcon
        vz=vz+0.5*dt*fz/vcon
        
        times.append(timestep)
        positions.append((x,y,z,r2))
        sys.stdout.flush()
        sys.stdout.write('\r'+'Done {0} steps.' .format(i))
    
    sys.stdout.write('\r' + ' '*50)
    
    #Final angular momenta
    amx=y*vz-z*vy
    amy=z*vx-x*vz
    amz=x*vy-y*vx
    am=np.sqrt(amx*amx+amy*amy+amz*amz)
    bamf = np.arcsin(amz/am)
    dbam = bamf - bam0
    lamf=np.arctan2(-amy,-amx)
    if lamf < 0.:
        lamf += 2.*np.pi
        #check to make sure have not crossed 0/2*pi
    if np.abs(lamf-lam0) > np.pi:
        if lam < np.pi/2.:
             lamf -= 2.*np.pi
        else:
             lamf += 2.*np.pi
    dlam=lamf-lam0
        
    #Final energy
    finalenergy = energyfunctions['X'](x, y, z, vx, vy, vz)
        
    return {'positions':np.array(positions),
            'initialenergy':np.array(initialenergy),
            'finalenergy':np.array(finalenergy), 
            'times':np.array(times), 
            'lam0':np.rad2deg(lam0),
            'bam0':np.rad2deg(bam0),
            'dlam':np.rad2deg(dlam),
            'dbam':np.rad2deg(dbam)}

## Functions to Get Orbital Parameters

In [165]:
def find_apo_peri(r2):
    id_pe = [i for i in np.arange(1, len(r2)-1) if r2[i-1] > r2[i] and r2[i+1] > r2[i]]
    id_ap = [i for i in np.arange(1, len(r2)-1) if r2[i-1] < r2[i] and r2[i+1] < r2[i]]
       
    apos = [r2[i] for i in id_ap]
    peris = [r2[i] for i in id_pe]
    apo_mean = np.sqrt(np.mean(apos))
    peri_mean = np.sqrt(np.mean(peris))

    return id_pe, id_ap, peri_mean, apo_mean

def find_ecc(r2, id_pe, id_ap):
    N_pe = len(id_pe)
    N_ap = len(id_ap)
    eccentricities = []
    for i in np.arange(0, min(N_pe,N_ap)):   
        e = (np.sqrt(r2[id_ap[i]]) - np.sqrt(r2[id_pe[i]]))/(np.sqrt(r2[id_ap[i]])+np.sqrt(r2[id_pe[i]]))
        eccentricities.append(e)
    return np.mean(eccentricities)

def find_period(times, id_ap): 
    #Find average period from R_a to R_a 
    periods = []
    for i in np.arange(1, len(id_ap)):
        period = times[id_ap[i]]-times[id_ap[i-1]]
        periods.append(period)
    return np.mean(periods) #Gyr

def pick3points(idpe,idap,index,x,y,z):
    #x, y, z are arrays, i.e. time series of x, y, z values
    if idpe[index] < idap[index]:
        #perigalacticon occurs first
        #Peri first
        point1 = (x[idpe[index]], y[idpe[index]], z[idpe[index]])
        
        #Apogalacticon last
        point3 = (x[idap[index]], y[idap[index]], z[idap[index]])
        
    else:
        #apogalacticon occurs first
        #Apogalacticon first
        point1 = (x[idap[index]], y[idap[index]], z[idap[index]])
        
        #Perigalacticon last
        point3 = (x[idpe[index]], y[idpe[index]], z[idpe[index]])

    #Point between the two
    indexbetween = (idpe[index]+idap[index])/2
    point2 = (x[indexbetween], y[indexbetween], z[indexbetween])
    return [point1, point2, point3]

def FINDPLANE(points): #points is list of three 3-tuples
    point1, point2, point3 = points
    
    #Vector p1p2 goes from p1 to p2, vector p1p3 goes from p1 to p3
    #The points are ordered in time, so galaxy reaches p1, then p2, then p3
    #Cross product p1p2 X p1p3 thus yields normal vector to 
    #orbital plane, this vector being parallel to the angular velocity vector, 
    #by right hand rule
    
    p1p2 = (point2[0]-point1[0], point2[1]-point1[1], point2[2]-point1[2]) 
    p1p3 = (point3[0]-point1[0], point3[1]-point1[1], point3[2]-point1[2])

    return np.cross(p1p2, p1p3) #Normal vector

def FINDINC(normalvector):
    #Normal vector is assumed to be parallel to angular velocity vector
    a, b, c = normalvector 
    
    #Inclination angle of orbit plane to galactic plane.
    #Zero is in galactic plane and orbiting in the direction of galactic rotation
    theta=np.arccos(-c/np.sqrt(a*a+b*b+c*c))
    return theta #radians!

def FINDLONGITUDE(normalvector):     
    #input is normal vector to orbital plane, parallel to angular velocity vector
    #longitude of ascending node measured in direction opposite galactic rotation from x=0
    a = normalvector[0]
    b = normalvector[1]
    omega=np.arctan2(b,a)+np.pi/2.0
    if omega < 0:
        omega += 2.*np.pi
    return omega

def find_lpole_bpole(theta, omega):
    bpole=theta-(np.pi/2.)
    lpole=omega+0.5*np.pi
    lpole = np.rad2deg(lpole)
    bpole = np.rad2deg(bpole)
    #if lpole > 2.*np.pi:
    #    lpole -= 2.*np.pi
    if lpole >= 0:
        lpole = lpole % 360
    else:
        lpole = lpole % -360
    if bpole >= 0:
        bpole = bpole % 360
    else:
        bpole = bpole % -360

    return lpole, bpole

def get_rt(r2, idpe, idap, ldsph=ldsph, mol1=mol1, mol2=mol2, d=distance, ratoam=ratoam):
    #Calculate the tidal radius at pericenter and store it
    N_pe = len(idpe)
    N_ap = len(idap)
    rt1 = []
    rt2 = []
    for i in np.arange(1, min(N_pe, N_ap)):
        r_ap = np.sqrt(r2[idap[i]])    
        r_pe=np.sqrt(r2[idpe[i]])
        ec=(r_ap-r_pe)/(r_ap+r_pe)
        fe=((1.-ec)**2)/((((1.+ec)**2)/(2.*ec))*np.log((1.+ec)/(1.-ec))+1.)
        mmw=1.1e10*(r_ap+r_pe)/2.
        rt1.append(((r_ap+r_pe)/2.)*(fe*mol1*ldsph/mmw)**0.333333)
        rt2.append(((r_ap+r_pe)/2.)*(fe*mol2*ldsph/mmw)**0.333333)
        
    rt1=np.mean(rt1)
    rt2=np.mean(rt2)
    rt1=np.arctan(rt1/d)*ratoam
    rt2=np.arctan(rt2/d)*ratoam
    
    return rt1, rt2

In [166]:
    #do some checking to avoid splitting omega values across 0/2pi
    #and the same for lam
    #     if (abs(lam(k)-lam(1)).gt.pi) then
    #        if (lam(1).lt.pi) then
    #           lam(k)=lam(k)-2.0d0*pi
    #        else
    #           lam(k)=lam(k)+2.0d0*pi
    #        end if
    #     end if

## Putting together the MC Sample Construction + Integration + Parameters

In [363]:
NMC = 5 #Number of MC experiments. 
t = 100 #Gyr
NSTEP = 400

#Get initial position
x0, y0, z0 = get_xyz(ra, dec, distance)

print "Constructing sample of {0} proper motions from Gaussian distributions of mu_a, mu_d, radial velocity..." \
.format(NMC)
mu_a_sample = np.random.normal(loc=mu_a, scale=mu_a_std, size=NMC-1)
mu_d_sample = np.random.normal(loc=mu_d, scale=mu_d_std, size=NMC-1)
rv_sample = np.random.normal(loc=rv, scale=rv_std, size=NMC-1)

pm_sample = np.column_stack([mu_a_sample, mu_d_sample, rv_sample])
pm_sample = np.insert(pm_sample, 0, (mu_a, mu_d, rv), axis=0) 
#Now the first value is a tuple with the measured or "actual" mu_a, mu_d, rv

print "Converting to sample of xyz velocities..."
velsample = [get_vxvyvz(ra, dec, distance, pm[0], pm[1], pm[2]) for pm in pm_sample]

#Make bunch of empty lists for our orbital parameters
apos, peris, es, periods, thetas, omegas, lpoles, bpoles, \
lam0s, bam0s, lamfs, bamfs, dlams, dbams, rt1s, rt2s = [[] for i in np.arange(16)]

Nfailed = 0
experiment = 0

print "Commencing MC experiments..."
for velocityvector in velsample:
    experiment += 1
    
    sys.stdout.write('\r'+'Doing experiment {0}' .format(experiment))# + '\n')
    time.sleep(2)
    sys.stdout.write('\r'+' '*50)

    vx0, vy0, vz0 = velocityvector
    result = integrate_orbit('X', x0, y0, z0, vx0, vy0, vz0, t=t, NSTEP=NSTEP)
    
    positions = result['positions']
    times = result['times']
    x = [pos[0] for pos in positions]
    y = [pos[1] for pos in positions]
    z = [pos[2] for pos in positions]
    r2 = [pos[3] for pos in positions]

    #Get apo and peri
    id_pe, id_ap, peri_mean, apo_mean = find_apo_peri(r2)
    N_pe = len(id_pe)
    N_ap = len(id_ap)
    if N_ap < 2:
        #If not at least one complete orbit from apo to apo, can't get parameters
        Nfailed += 1
        break
    apos.append(apo_mean)
    peris.append(peri_mean)

     
    #Get eccentr
    ecc = find_ecc(r2, id_pe, id_ap)
    es.append(ecc)
    
    #Get period
    avg_period = find_period(times, id_ap)
    periods.append(avg_period)
    
    #Get Inclination and Longitude
    setsofpoints = [pick3points(id_pe, id_ap, i, x, y, z) for i in np.arange(1,min(N_pe,N_ap))]
    inclinations = [FINDINC(FINDPLANE(pointset)) for pointset in setsofpoints]
    longitudes = [FINDLONGITUDE(FINDPLANE(pointset)) for pointset in setsofpoints]
    avg_theta = np.mean(inclinations)
    avg_omega = np.mean(longitudes)
    thetas.append(np.rad2deg(avg_theta))
    omegas.append(np.rad2deg(avg_omega))
    #print "inclination, longitude =", np.rad2deg(avg_theta), np.rad2deg(avg_omega)
    
    #SAME THING AS ABOVE
    #for i in np.arange(1,min(N_pe,N_ap)):
    #points = pick3points(id_pe, id_ap, i, test_x, test_y, test_z)
    #normalvector = FINDPLANE(points)       
    #theta = FINDINC(normalvector)
    #omega = FINDLONGITUDE(normalvector)
    #print normalvector, theta, omega
    
    #Get lpole, bpole
    lpole, bpole = find_lpole_bpole(avg_theta, avg_omega)
    lpoles.append(lpole)
    bpoles.append(bpole)
    #print "lpole, bpole =", lpole, bpole
    
    #Get initial angular momentum
    lam0 = result['lam0']
    bam0 = result['bam0']
    lam0s.append(lam0)
    bam0s.append(bam0)
    #print "lam0, bam0 =", lam0, bam0
    
    #Get rt1, rt2
    rt1, rt2 = get_rt(r2, id_pe, id_ap) 
    rt1s.append(rt1)
    rt2s.append(rt2)
    #print "rt1, 2 =", rt1, rt2
    
    #dlam, dbam
    dlams.append(result['dlam'])
    dbams.append(result['dbam'])

#Initial velocity
vx0, vy0, vz0 = velsample[0]
    
#Initial angular momentum
amx0=y0*vz0-z0*vy0
amy0=z0*vx0-x0*vz0
amz0=x0*vy0-y0*vx0
    
print "\n", "l, b, distance:", l, b, distance
print "x0,y0,z0:", x0, y0, z0   
print "vx0,vy0,vz0:", vx0, vy0, vz0
print "amx0,amy0,amz0:", amx0, amy0, amz0   
print "lam0, bam0:", lam0s[0], bam0s[0], "\n"  

print "Actual Perigalacticon (kpc) :", peris[0] 
print "Mean Perigalacticon (kpc) :", np.mean(peris), "+-", np.std(peris)
print "Max Perigalacticon (kpc) :", np.amax(peris)
print "Min Perigalacticon (kpc) :", np.amin(peris)
lower, upper = st.t.interval(0.95, len(peris)-1, loc=np.mean(peris), scale=st.sem(peris))
print "95% Confidence Limits (kpc) :", lower, upper, '\n'

print "Actual Apogalacticon (kpc) :", apos[0] 
print "Mean Apogalacticon (kpc) :", np.mean(apos), "+-", np.std(apos)
print "Max Apogalacticon (kpc) :", np.amax(apos)
print "Min Apogalacticon (kpc) :", np.amin(apos)
lower, upper = st.t.interval(0.95, len(apos)-1, loc=np.mean(apos), scale=st.sem(apos))
print "95% Confidence Limits (kpc) :", lower, upper, '\n'

print "Actual Eccentricity :", es[0] 
print "Mean Eccentricity :", np.mean(es), "+-", np.std(es)
print "Max Eccentricity :", np.amax(es)
print "Min Eccentricity :", np.amin(es)
lower, upper = st.t.interval(0.95, len(es)-1, loc=np.mean(es), scale=st.sem(es))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual Orbital Period (Gyr) :", periods[0] 
print "Mean Orbital Period (Gyr) :", np.mean(periods), "+-", np.std(periods)
print "Max Orbital Period (Gyr) :", np.amax(periods)
print "Min Orbital Period (Gyr) :", np.amin(periods)
lower, upper = st.t.interval(0.95, len(periods)-1, loc=np.mean(periods), scale=st.sem(periods))

print "95% Confidence Limits (Gyr) :", lower, upper, '\n'

print "Actual r_t1 (arcmin) :", rt1s[0] 
print "Mean r_t1 (arcmin) :", np.mean(rt1s), "+-", np.std(rt1s)
print "Max r_t1 (arcmin) :", np.amax(rt1s)
print "Min r_t1 (arcmin) :", np.amin(rt1s)
lower, upper = st.t.interval(0.95, len(rt1s)-1, loc=np.mean(rt1s), scale=st.sem(rt1s))

print "95% Confidence Limits (arcmin) :", lower, upper, '\n'

print "Actual r_t2 (arcmin) :", rt2s[0] 
print "Mean r_t2 (arcmin) :", np.mean(rt2s), "+-", np.std(rt2s)
print "Max r_t2 (arcmin) :", np.amax(rt2s)
print "Min r_t2 (arcmin) :", np.amin(rt2s)
lower, upper = st.t.interval(0.95, len(rt2s)-1, loc=np.mean(rt2s), scale=st.sem(rt2s))

print "95% Confidence Limits (arcmin) :", lower, upper, '\n'

print "Actual Inclination (degrees) :", thetas[0] 
print "Mean Inclination (degrees) :", np.mean(thetas), "+-", np.std(thetas)
print "Max Inclination (degrees) :", np.amax(thetas)
print "Min Inclination (degrees) :", np.amin(thetas)
lower, upper = st.t.interval(0.95, len(thetas)-1, loc=np.mean(thetas), scale=st.sem(thetas))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual Longitude (degrees) :", omegas[0] 
print "Mean Longitude (degrees) :", np.mean(omegas), "+-", np.std(omegas)
print "Max Longitude (degrees) :", np.amax(omegas)
print "Min Longitude (degrees) :", np.amin(omegas)
lower, upper = st.t.interval(0.95, len(omegas)-1, loc=np.mean(omegas), scale=st.sem(omegas))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual lpole (degrees) :", lpoles[0] 
print "Mean lpole (degrees) :", np.mean(lpoles), "+-", np.std(lpoles)
print "Max lpole (degrees) :", np.amax(lpoles)
print "Min lpole (degrees) :", np.amin(lpoles)
lower, upper = st.t.interval(0.95, len(lpoles)-1, loc=np.mean(lpoles), scale=st.sem(lpoles))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual bpole (degrees) :", bpoles[0] 
print "Mean bpole (degrees) :", np.mean(bpoles), "+-", np.std(bpoles)
print "Max bpole (degrees) :", np.amax(bpoles)
print "Min bpole (degrees) :", np.amin(bpoles)
lower, upper = st.t.interval(0.95, len(bpoles)-1, loc=np.mean(bpoles), scale=st.sem(bpoles))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual lam (degrees) :", lam0s[0] 
print "Mean lam (degrees) :", np.mean(lam0s), "+-", np.std(lam0s)
print "Max lam (degrees) :", np.amax(lam0s)
print "Min lam (degrees) :", np.amin(lam0s)
lower, upper = st.t.interval(0.95, len(lam0s)-1, loc=np.mean(lam0s), scale=st.sem(lam0s))

print "95% Confidence Limits :", lower, upper, '\n'

print "Actual bam (degrees) :", bam0s[0] 
print "Mean bam (degrees) :", np.mean(bam0s), "+-", np.std(bam0s)
print "Max bam (degrees) :", np.amax(bam0s)
print "Min bam (degrees) :", np.amin(bam0s)
lower, upper = st.t.interval(0.95, len(bam0s)-1, loc=np.mean(bam0s), scale=st.sem(bam0s))
print "95% Confidence Limits :", lower, upper, '\n'

print "Average Change in lam, bam (degrees) :", np.nanmean(dlams), np.nanmean(dbams), '\n'

print "Number of MC Experiments :", NMC
print "Number of Successful Experiments :", NMC - Nfailed

Constructing sample of 5 proper motions from Gaussian distributions of mu_a, mu_d, radial velocity...
Converting to sample of xyz velocities...
Commencing MC experiments...
                                                  
l, b, distance: 220.16913737 67.2312443585 233
x0,y0,z0: 77.1058750111 58.1663853521 214.843319663
vx0,vy0,vz0: 41.2271830735 -115.559076583 40.498068962
amx0,amy0,amz0: 27182.7219156 5734.74582831 -11308.3199331
lam0, bam0: 191.913012306 -22.1488456026 

Actual Perigalacticon (kpc) : 163.332697601
Mean Perigalacticon (kpc) : 150.229133574 +- 63.2788751268
Max Perigalacticon (kpc) : 225.073986465
Min Perigalacticon (kpc) : 38.7641624442
95% Confidence Limits (kpc) : 62.3839720198 238.074295128 

Actual Apogalacticon (kpc) : 245.551171912
Mean Apogalacticon (kpc) : 368.660920693 +- 204.309499304
Max Apogalacticon (kpc) : 771.811675312
Min Apogalacticon (kpc) : 242.131058416
95% Confidence Limits (kpc) : 85.0338660494 652.287975337 

Actual Eccentricity : 0.2010798886

In [398]:
import scipy.stats as st

a = [1,5,3.4,14, 32.3, 19, 5, 67, 80, 45.6, 98, 43, 67, 55, 10]
npa = np.array(a)
wheres = np.where(np.array(a) > 50)

#print st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))
#print sms.DescrStatsW(a).tconfint_mean()

#print np.mean(a), np.std(a), len(a)

In [409]:
import cosmolopy.distance as cd
import cosmolopy.constants as cc

def L_nu_from_magAB(magAB):
    """Convert absolute magnitude into luminosity (erg s^-1 Hz^-1)."""
    const = 4. * math.pi * (10. * cc.pc_cm)**2.
    L_nu =  const * 10.**((magAB + 48.6)/(-2.5))
    return L_nu

ImportError: No module named cosmolopy.distance

In [439]:
def absmag_to_L(M_object):
    #Convert absolute magnitude to luminosity in units of solar luminosities
    M_sun = 4.83 #absolute magnitude of sun
    L_object = 10**(.4*(M_sun-M_object))
    
    #L_0 = 3.0128e28 #Watts
    #L_sun = 3.827e26 #Watts
    #L_object = L_0*(10**(-.4*M_object))
    
    return L_object

absmag_to_L(-9.8)

#6.54e6

711213.5136533296

In [437]:
def linear_to_angular_size(linear_size, distance):
    #Returns angular size in arcminutes
    linear_size=float(linear_size)
    theta = np.arctan(linear_size/distance) #Radians
    return theta * 3437.75 #Arcmin

In [459]:
linear_to_angular_size(176, 233000)

2.5967548709249471

In [456]:
print np.arange(0, 1)

[0]


In [468]:
print np.rad2deg(np.arctan2(-1, -1))
print np.rad2deg(np.arctan2(-1, -1) + 2*np.pi)

-135.0
225.0


In [470]:
print np.arctan2(-1, -1)
print np.arctan2(-1, -1) + 2.*np.pi

-2.35619449019
3.92699081699


In [481]:
vec = (1, 1, -1)
FINDINC(vec)

0.95531661812450919

In [490]:
print 730 % 360
print -730 % 360

10
350
